In [6]:
import pandas as pd

from datetime import datetime

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

In [7]:
class MucuriModel:
    def __init__(self):

        self.model = None
        self._build_model()

    def _build_model(self):

        if self.model is None:
            model = Sequential()
            model.add(Dense(9, input_shape=(9,)))
            model.add(Dense(9, activation="tanh"))
            model.add(Dense(6, activation="tanh"))
            model.add(Dense(1, activation="linear"))
            model.compile(loss="mean_squared_error", optimizer="adam")

    def train(self, X, Y, X_test=None, Y_test=None):

        assert self.model is not None

        checkpoint = ModelCheckpoint(filepath="weights.hdf5", save_best_only=True)
        self.model.fit(
            X,
            Y,
            verbose=1,
            epochs=30,
            validation_data=(X_test, Y_test)
            if X_test is not None and Y_test is not None
            else None,
            callbacks=[checkpoint],
        )

    def predict(self, data):

        assert self.model is not None

        return self.model.predict(data)


In [8]:
f = pd.ExcelFile("./Mucuri_novo_semNaN_torre150m.xlsx")

df = f.parse("Dados anemo")

_train_data_1 = df[pd.to_datetime(df["Data"]) <= datetime(year=2015, month=12, day=22)]
_train_data_2 = df[
    (pd.to_datetime(df["Data"]) == datetime(year=2015, month=12, day=23))
    & (df["hora"] <= 11)
]

train_data = pd.concat([_train_data_1, _train_data_2])

_test_data_1 = df[
    (pd.to_datetime(df["Data"]) == datetime(year=2015, month=12, day=23))
    & (df["hora"] >= 12)
]
_test_data_2 = df[
    (pd.to_datetime(df["Data"]) >= datetime(year=2015, month=12, day=24))
    & (pd.to_datetime(df["Data"]) <= datetime(year=2015, month=12, day=30))
]
_test_data_3 = df[
    (pd.to_datetime(df["Data"]) == datetime(year=2015, month=12, day=31))
    & (df["hora"] <= 13)
]

test_data = pd.concat([_test_data_1, _test_data_2, _test_data_3])

In [13]:
train_data

,Data,dia,mês,ano,hora,v_anemo2,dir_1,temp,umid,pressão
0,2015-11-30,30,11,2015,14,13.012139,75.105481,27.516129,72.930636,1020.422601
1,2015-11-30,30,11,2015,15,12.726087,68.334332,27.238095,75.212121,1020.394348
2,2015-11-30,30,11,2015,16,12.081111,64.457865,27.105263,75.741379,1020.508333
3,2015-11-30,30,11,2015,17,11.647222,53.842100,26.305556,75.302632,1020.611000
4,2015-11-30,30,11,2015,18,11.064444,53.945279,25.464286,76.592593,1020.866500
...,...,...,...,...,...,...,...,...,...,...
545,2015-12-23,23,12,2015,7,2.912222,67.426924,22.637795,86.600000,1015.316167
546,2015-12-23,23,12,2015,8,3.958333,84.999251,22.455696,82.110390,1015.975667
547,2015-12-23,23,12,2015,9,5.216667,107.133454,22.602410,79.801205,1016.323667
548,2015-12-23,23,12,2015,10,6.506667,110.979895,22.203252,80.907407,1016.071500


In [14]:
test_data

,Data,dia,mês,ano,hora,v_anemo2,dir_1,temp,umid,pressão
550,2015-12-23,23,12,2015,12,12.242778,77.442821,21.516779,87.861111,1014.845833
551,2015-12-23,23,12,2015,13,9.268333,73.171002,23.075269,73.338889,1014.793333
552,2015-12-23,23,12,2015,14,8.734444,96.238878,22.545455,78.533333,1014.085333
553,2015-12-23,23,12,2015,15,9.662222,110.156291,21.459770,83.722222,1013.541500
554,2015-12-23,23,12,2015,16,8.620556,97.686662,22.133803,84.691275,1013.563333
...,...,...,...,...,...,...,...,...,...,...
739,2015-12-31,31,12,2015,9,7.370556,88.967009,25.545455,90.612500,1018.871167
740,2015-12-31,31,12,2015,10,10.400556,89.474475,23.558442,91.142857,1018.303667
741,2015-12-31,31,12,2015,11,12.400556,89.119129,23.369863,90.567376,1017.004667
742,2015-12-31,31,12,2015,12,13.944444,85.945816,23.240741,88.645833,1016.574167
